# Compute the metrics and write it to TEX file

In [ ]:
import glob
import numpy as np
import torch as T
import pickle

In [ ]:
file = open('sym.tex','w')
file.write('\\begin{table*}[t]\n')
file.write('\t\\caption{Results of speaker independent model}\n')
file.write('\t\\label{tab:results_main}\n')
file.write('\t\\centering\n')
file.write('\t\\begin{tabular}{l |ccc|ccc| ccc}\n')
file.write('\t\t\\toprule\n')
file.write('\t\t\\bf Model & \\multicolumn{3}{c}{\\bf ROLL} & \\multicolumn{3}{c}{\\bf PITCH} & \\multicolumn{3}{c}{\\bf YAW} \\\\\n')
file.write('\t\t  & \\bf MAE & \\bf CC & \\bf p-val & \\bf MAE & \\bf CC & \\bf p-val & \\bf MAE & \\bf CC & \\bf p-val\\\\\n')
file.write('\t\t\\midrule\n')

In [ ]:
def _gaussian_kernel(size, sigma):
    #interval = (2 * sigma**2) / (size - 1)
    x = T.arange(size, dtype=T.float32)
    kernel = T.exp(-((x - size // 2) ** 2) / (2 * sigma ** 2))
    return kernel / T.sum(kernel)

In [ ]:
from scipy.stats import pearsonr
import sys
dirs = glob.glob('results/*')
kernel = _gaussian_kernel(10, 3).unsqueeze(0).unsqueeze(0).expand(3,-1,-1)
print(kernel)
for dirw in dirs:
    print(dirw)
    sym = pickle.load(open(dirw,'rb'))
    input = sym['input']
    output = sym['output']
    cat = zip(input,output)
    mae = np.array([0,0,0],dtype=np.float64)
    std = np.array([0,0,0],dtype=np.float64)
    corr = []
    pval = []
    for inp,out in cat:
        inp = inp.squeeze()
        out = out.squeeze()
        out = np.array(T.nn.functional.conv1d(T.tensor(out).unsqueeze(0).transpose(1,2), kernel, groups=3, padding='same').transpose(1,2)).squeeze()
        err = np.array(out-inp)
        ae = np.abs(err)
        std += np.std(err,0)
        #print(np.mean(ae,0))
        mae += np.mean(ae,0)
        cor = [0,0,0]
        pva = [0,0,0]
        for j in range(3):
            try:
                cor[j], pva[j]= pearsonr(inp[:,j],out[:,j])
            except:
                print(out[:,j])
                sys.exit()
        corr.append(cor)
        pval.append(pva)
    mae = mae/len(input)
    std = std/len(input)
    corr = np.mean(np.stack(corr,0),0)
    pval = np.mean(np.stack(pval,0),0)
    file.write('\t\t%20s & $%.2f\\pm%.2f$ & $%.2f$ & $%.2f$'%(sym['name'],mae[1],std[1],corr[1],pval[1]))
    file.write(' & $%.2f\\pm%.2f$ & $%.3f$ & $%.3f$'%(mae[0],std[0],corr[0],pval[0]))
    file.write(' & $%.2f\\pm%.2f$ & $%.3f$ & $%.3f$\\\\\n'%(mae[2],std[2],corr[2],pval[2]))
    print('%20s:[roll,pitch,yaw]:\t[%.4f±%.4f,%.4f±%.4f,%.4f±%.4f], \t[mean]:[%.4f±%.4f]'%(sym['name'],mae[1],std[1],mae[0],std[0],mae[2],std[2],np.mean(mae),np.mean(std)))
    print('%20s: \troll:[corr:%.4f, conf:%.4f], \tpitch:[corr:%.4f, conf:%.4f], \tyaw:[corr:%.4f, conf:%.4f], \tmean:%.4f'%(sym['name'],corr[1],pval[1],corr[0],pval[0],corr[2],pval[2],np.mean(corr)))

In [ ]:
file.write('\t\t\\bottomrule\n')
file.write('\t\\end{tabular}\n')
file.write('\\end{table*}')
file.close()